In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import math
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import Kan_NN
import importlib
importlib.reload(Kan_NN)

<module 'Kan_NN' from 'c:\\Users\\JP\\Documents\\TU Berlin\\Master\\Code_clean\\Kan_NN.py'>

In [3]:
def compute_sum(n, d):
    total_sum = 0
    for w in range(1, d + 1):
        binom_n_w = math.comb(n, w)  # Compute binomial coefficient C(n, w)
        binom_d_w = math.comb(d, w)  # Compute binomial coefficient C(d, w)
        total_sum += (2 ** w) * binom_n_w * binom_d_w  # Add the term to the sum
    return total_sum

In [4]:
##### Problem
in_dim = 12
degree = 2
shape = [in_dim, compute_sum(in_dim, degree), 1]
h = [16]
Batch_size = 512
print(shape)


[12, 312, 1]


In [5]:
import itertools
class RandomPolynomial(torch.nn.Module):
    def __init__(self, input_dim, degree):
        super(RandomPolynomial, self).__init__()
        self.input_dim = input_dim
        self.degree = degree
        self.exponents = self.generate_exponents(degree, input_dim)
        torch.manual_seed(42)
        self.coefficients = torch.randn(len(self.exponents))
        print(self.exponents)
        print(self.coefficients)

    def forward(self, X):
        batch_size = X.shape[0]
        result = torch.zeros(batch_size, 1)

        # For each monomial (term) in the polynomial, compute the corresponding product of powers of X
        for idx, exp in enumerate(self.exponents):
            term = torch.ones(batch_size, 1)
            for dim, e in enumerate(exp):
                term *= X[:, dim].pow(e).view(-1, 1)
            result += self.coefficients[idx] * term
        return result

    def generate_exponents(self, degree, input_dim):
        exponents = []
        for exp_tuple in itertools.product(range(degree + 1), repeat=input_dim):
            if sum(exp_tuple) <= degree:
                exponents.append(exp_tuple)
        return exponents
    
poly = RandomPolynomial(input_dim=in_dim, degree=degree)


[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0), (0, 0, 0, 0

In [6]:
import time
models = []
params = [] 
max_depth = 1
max_width = 1
for depth in range(1,max_depth):
    mods = []
    pars = []
    for i in range(1,max_width):
        model = Kan_NN.Neural_Kan(shape = shape, h = [2**i] * depth)
        print(model.params)
        if model.params > 131072:
            continue
        epochs = 1
        pars.append(model.params)
        train_loss = []
        model.train()
        optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.)
        criterion = torch.nn.MSELoss()
        for epoch in range(epochs):
            running_loss = 0.0 
            for samples in range(10):
                start_time = time.time()
                optimizer.zero_grad()
                input = torch.randn(Batch_size, in_dim)
                target = poly(input)
                outputs = model(input)
                loss = criterion(target, outputs)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            avg_loss = running_loss / 100
            train_loss.append(avg_loss)
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.6f}, lr: {optimizer.param_groups[0]['lr']:6f}")
        mods.append(model)
        plt.plot(train_loss)
        plt.title(f'train_loss on arbitary polynomial h = {[2**i] * depth}')
        plt.legend()
        plt.show()
        print("Training Complete!")
    models.append(mods)
    params.append(pars)


In [7]:
import dill
with open("polys_KAN.dill", "wb") as f:
    dill.dump(models, f)